In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.options.display.max_columns=100

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

In [ ]:
url=f"https://www.iqair.com/world-most-polluted-cities?sort=-rank&page=1&perPage=50&cities="

html=requests.get(url,headers=headers).content
soup=BeautifulSoup(html)

max_pages=int(soup.select_one('app-table-pagination-pages').text.strip().split(' ')[-1])

In [ ]:
all_cities_df=pd.DataFrame()

for page in range(1,max_pages+1):
    try:
        url=f"https://www.iqair.com/world-most-polluted-cities?sort=-rank&page={page}&perPage=50&cities="
        html=requests.get(url,headers=headers).content
        page_df=pd.read_html(html)[0]
        all_cities_df=pd.concat([all_cities_df,page_df])
    except:
        continue

    print(f'{page}/{max_pages}',end='\r')

all_cities_df.to_csv('all_cities_df.csv',index=False)


In [ ]:
all_cities_df=pd.read_csv('all_cities_df.csv')

In [ ]:
import osmnx
import pickle

lat_lon_dict=pickle.load(open('lat_lon_dict.pickle','rb'))

for i, city in enumerate(all_cities_df['City'].unique(),start=1):
    if city in lat_lon_dict:
        continue

    try:
        location = osmnx.geocoder.geocode(city)
        lat, lon = location[0], location[1]
        lat_lon_dict[city]={'Lat':lat,'Lon':lon}
    except:
        continue

    print(f'Progress: {i}/{all_cities_df['City'].nunique()}',end='\r')

    if i%100==0 or i==all_cities_df['City'].nunique():
        pickle.dump(lat_lon_dict,open('lat_lon_dict.pickle','wb'))




In [ ]:
import pickle
lat_lon_dict=pickle.load(open('lat_lon_dict.pickle','rb'))

In [ ]:
coordinates_df=pd.DataFrame.from_dict(lat_lon_dict,orient='index').reset_index(names='City')
all_cities_df=pd.read_csv('all_cities_df.csv')
all_cities_df=all_cities_df.merge(coordinates_df,on='City')
all_cities_df[all_cities_df.columns[2:]]=all_cities_df[all_cities_df.columns[2:]].apply(pd.to_numeric,errors='coerce')
all_cities_df=all_cities_df[['City', 'Lat', 'Lon', 'Rank', '2017', '2018', '2019', '2020', '2021', '2022', '2023']]



all_cities_df=pd.melt(
    all_cities_df,
    id_vars=["City", "Lat", "Lon", "Rank"],  # Columns to keep
    value_vars=[str(year) for year in range(2017, 2024)],  # Columns to melt
    var_name="Year",  # New column name for years
    value_name="AQI"  # New column name for values
)

all_cities_df=all_cities_df.dropna(how='any')




all_cities_df.to_csv('all_cities_df (cleaned).csv',index=False)

In [12]:
all_cities_df=pd.read_csv('all_cities_df (cleaned).csv')